In [3]:
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
import dataframe_image as dfi



def create_ranking_table(t,file_name,distinct=False):
    models = ['Linear','KNN','G-Boost','MLP']
    names = ['Linear*','KNN*','G-Boost*','MLP*']
    rankings = None
    mae_baseline = []
    for i,model in enumerate(models):
        file = "../data/ranking_NPI/" + model + t +".csv"
        df = pd.read_csv(file,index_col=0)
        mae_baseline.append(df['mae_baseline'].values[0])
        if distinct:
            df = df.rename(columns={'feature_asym':'feature'})
        else:
            df['feature'] = df.apply(lambda row: row.feature_asym.replace('_i','').replace('_j',''), axis=1)
            df = df.groupby(['feature']).agg({'importance_mean': ['mean'], 'importance_std': ['mean']}).reset_index()
        if rankings is None:
            rankings = pd.DataFrame(df['importance_mean'].values,index=df['feature'],columns=[names[i]])
        else:
            rankings[names[i]+t] = df['importance_mean'].values
    
    if distinct :
        rankings = rankings.drop(index=['date','pair_id'])
    else:
        rankings = rankings.drop(index=['date','paird'])
    rankings.columns = ['Linear*','KNN*','G-Boost*','MLP*']

    rankings[rankings.columns] = MinMaxScaler().fit_transform(rankings[rankings.columns])
    rankings['avg_importance'] = rankings.values.mean(axis=1)
    rankings = rankings.sort_values(by='avg_importance', axis=0,ascending=False)

    rankings = rankings.rename(index={"new_cases_per_million":"New Covid cases",
    "new_deaths_per_million":"New Covid deaths",
    "new_cases_per_million_7":"New Covid cases -7",
    "new_deaths_per_million_7":"New Covid deaths -7",
    "new_cases_per_million_14":"New Covid cases -14",
    "new_deaths_per_million_14":"New Covid deaths -14",
    "c1_schoolclosing":"C1 School closing",
    "c2_workplaceclosing":"C2 Workplace closing",
    "c3_cancel_events":"C3 Cancel public events",
    "c4_restr_gather":"C4 Restrictions gatherings",
    "c5_closepublictransport":"C5 Close public transport",
    "c6_stay_home":"C6 Stay home requirements",
    "c7_restrnternal_move":"C7 Restr. Internal movement",
    "c8nt_trvl_controls":"C8 Inter. travel controls",
    "h2_testingpolicy":"H2 Testing policy",
    "h3_contacttracing":"H3 Contact tracing"})
    rankings = rankings.rename(index={"new_cases_per_million_i":"New Covid cases O",
    "new_deaths_per_million_i":"New Covid deaths O",
    "new_cases_per_million_i_7":"New Covid cases -7 O",
    "new_deaths_per_million_i_7":"New Covid deaths -7 O",
    "new_cases_per_million_i_14":"New Covid cases -14 O",
    "new_deaths_per_million_i_14":"New Covid deaths -14 O",
    "c1_schoolclosing_i":"C1 School closing O",
    "c2_workplaceclosing_i":"C2 Workplace closing O",
    "c3_cancel_events_i":"C3 Cancel public events O",
    "c4_restr_gather_i":"C4 Restrictions gatherings O",
    "c5_closepublictransport_i":"C5 Close public transport O",
    "c6_stay_home_i":"C6 Stay home requirements O",
    "c7_restr_internal_move_i":"C7 Restr. Internal movement O",
    "c8_int_trvl_controls_i":"C8 Inter. travel controls O",
    "h2_testingpolicy_i":"H2 Testing policy O",
    "h3_contacttracing_i":"H3 Contact tracing O",
    "new_cases_per_million_j":"New Covid cases D",
    "new_deaths_per_million_j":"New Covid deaths D",
    "new_cases_per_million_j_7":"New Covid cases -7 D",
    "new_deaths_per_million_j_7":"New Covid deaths -7 D",
    "new_cases_per_million_j_14":"New Covid cases -14 D",
    "new_deaths_per_million_j_14":"New Covid deaths -14 D",
    "c1_schoolclosing_j":"C1 School closing D",
    "c2_workplaceclosing_j":"C2 Workplace closing D",
    "c3_cancel_events_j":"C3 Cancel public events D",
    "c4_restr_gather_j":"C4 Restrictions gatherings D",
    "c5_closepublictransport_j":"C5 Close public transport D",
    "c6_stay_home_j":"C6 Stay home requirements D",
    "c7_restr_internal_move_j":"C7 Restr. Internal movement D",
    "c8_int_trvl_controls_j":"C8 Inter. travel controls D",
    "h2_testingpolicy_j":"H2 Testing policy D",
    "h3_contacttracing_j":"H3 Contact tracing D"})
    rankings = rankings.sort_values(by='avg_importance', axis=0,ascending=False)
    
    rankings.to_csv('../data/ranking_NPI/' + file_name + '.csv')
    
    
    styler = rankings.style
    styler.set_properties(subset=rankings.columns, **{'width': '80px'})
    styler.set_precision(2)

    styler.background_gradient(cmap='plasma',vmin=rankings.min().min(),vmax=rankings.max().max())
    styler.format("{:.2%}")
    dfi.export(styler,'images/' + file_name + '.png',fontsize=40)
    print(file_name.replace('_',' '))
    display(styler)

In [4]:
create_ranking_table('_pair_id',"Feature_importance_pair_id_distinct",distinct=True)

create_ranking_table('_date_pair_id',"Feature_importance_date_pair_id_distinct",distinct=True)

Feature importance pair id distinct


,Linear*,KNN*,G-Boost*,MLP*,avg_importance
feature,,,,,
C1 School closing O,100.00%,89.79%,100.00%,51.38%,85.29%
C1 School closing D,88.73%,75.49%,53.32%,54.25%,67.95%
New Covid cases O,13.68%,64.81%,48.00%,100.00%,56.62%
C3 Cancel public events D,36.20%,65.56%,71.89%,52.62%,56.57%
C7 Restr. Internal movement O,1.65%,80.41%,31.29%,90.27%,50.90%
C3 Cancel public events O,7.45%,100.00%,34.18%,41.82%,45.86%
C6 Stay home requirements D,31.59%,21.07%,96.37%,30.41%,44.86%
C4 Restrictions gatherings O,22.07%,60.62%,46.26%,48.93%,44.47%
H2 Testing policy O,46.37%,8.23%,83.92%,32.08%,42.65%


Feature importance date pair id distinct


,Linear*,KNN*,G-Boost*,MLP*,avg_importance
feature,,,,,
C1 School closing O,100.00%,68.83%,100.00%,58.75%,81.90%
C1 School closing D,93.74%,59.73%,35.86%,84.61%,68.49%
C3 Cancel public events D,19.12%,64.50%,76.55%,67.60%,56.94%
C3 Cancel public events O,10.04%,100.00%,33.19%,65.26%,52.12%
C7 Restr. Internal movement O,0.00%,92.97%,15.50%,100.00%,52.12%
H2 Testing policy O,14.10%,6.71%,87.22%,92.45%,50.12%
C4 Restrictions gatherings O,50.09%,51.07%,39.91%,53.88%,48.74%
C6 Stay home requirements D,16.94%,15.01%,96.18%,53.74%,45.47%
C6 Stay home requirements O,91.73%,15.61%,12.06%,59.45%,44.71%


In [5]:
create_ranking_table('_pair_id',"Feature_importance_pair_id",distinct=False)

create_ranking_table('_date_pair_id',"Feature_importance_date_pair_id",distinct=False)

Feature importance pair id


,Linear*,KNN*,G-Boost*,MLP*,avg_importance
feature,,,,,
C1 School closing,100.00%,99.83%,86.07%,86.05%,92.99%
C3 Cancel public events,20.71%,100.00%,59.54%,76.05%,64.08%
New Covid cases,10.99%,75.85%,63.39%,86.75%,59.24%
C4 Restrictions gatherings,9.02%,76.18%,64.62%,64.05%,53.47%
C7 Restr. Internal movement,1.44%,83.67%,27.33%,100.00%,53.11%
New Covid deaths,0.00%,76.61%,100.00%,0.00%,44.15%
C6 Stay home requirements,25.35%,22.90%,63.92%,38.23%,37.60%
H2 Testing policy,25.87%,1.75%,49.13%,30.18%,26.73%
C2 Workplace closing,2.34%,45.58%,22.52%,32.92%,25.84%


Feature importance date pair id


,Linear*,KNN*,G-Boost*,MLP*,avg_importance
feature,,,,,
C1 School closing,100.00%,78.11%,98.49%,90.52%,91.78%
C3 Cancel public events,14.47%,100.00%,79.43%,83.65%,69.39%
C6 Stay home requirements,55.79%,18.45%,78.33%,70.79%,55.84%
C7 Restr. Internal movement,6.05%,96.65%,19.60%,100.00%,55.57%
C4 Restrictions gatherings,28.47%,60.13%,79.82%,48.54%,54.24%
H2 Testing policy,11.06%,4.19%,64.35%,63.54%,35.78%
New Covid deaths,2.67%,10.31%,100.00%,0.00%,28.25%
C8 Inter. travel controls,0.00%,7.83%,47.17%,43.53%,24.63%
H3 Contact tracing,18.04%,19.89%,20.18%,34.71%,23.21%


In [6]:
create_ranking_table('_pair_id_PCA_2',"Feature_importance_pair_id_PCA_distinct",distinct=True)

create_ranking_table('_date_pair_id_PCA_2',"Feature_importance_date_pair_id_PCA_distinct",distinct=True)

Feature importance pair id PCA distinct


,Linear*,KNN*,G-Boost*,MLP*,avg_importance
feature,,,,,
Component_0_j,100.00%,100.00%,100.00%,100.00%,100.00%
Component_0_i,1.84%,78.85%,19.49%,91.65%,47.96%
Component_1_i,8.36%,40.72%,16.76%,38.07%,25.98%
New Covid cases O,0.46%,0.00%,17.51%,54.56%,18.13%
Component_1_j,0.74%,36.10%,0.00%,32.73%,17.39%
New Covid deaths D,2.00%,1.13%,7.82%,21.45%,8.10%
New Covid deaths O,0.11%,2.12%,2.13%,22.63%,6.75%
New Covid cases D,0.00%,0.47%,1.74%,0.00%,0.55%


Feature importance date pair id PCA distinct


,Linear*,KNN*,G-Boost*,MLP*,avg_importance
feature,,,,,
Component_0_j,100.00%,100.00%,100.00%,100.00%,100.00%
Component_0_i,13.32%,74.64%,20.31%,49.37%,39.41%
Component_1_i,1.15%,54.53%,17.36%,19.21%,23.06%
Component_1_j,8.52%,49.56%,0.00%,0.00%,14.52%
New Covid cases O,0.00%,0.00%,18.01%,29.62%,11.91%
New Covid deaths D,5.45%,14.18%,6.45%,11.03%,9.28%
New Covid deaths O,0.59%,12.67%,2.83%,8.63%,6.18%
New Covid cases D,0.12%,2.22%,0.51%,5.50%,2.09%


In [7]:
create_ranking_table('_pair_id_lags',"Feature_importance_pair_id_lags")

create_ranking_table('_date_pair_id_lags',"Feature_importance_date_pair_id_lags")

Feature importance pair id lags


,Linear*,KNN*,G-Boost*,MLP*,avg_importance
feature,,,,,
C1 School closing,100.00%,83.45%,79.11%,97.02%,89.89%
C3 Cancel public events,18.84%,100.00%,58.17%,76.55%,63.39%
C4 Restrictions gatherings,5.22%,74.08%,100.00%,31.81%,52.77%
C7 Restr. Internal movement,3.71%,73.57%,20.25%,100.00%,49.38%
C6 Stay home requirements,27.11%,23.98%,73.55%,30.73%,38.84%
H2 Testing policy,20.29%,2.37%,67.31%,28.88%,29.71%
New Covid cases,4.09%,40.88%,0.00%,67.89%,28.21%
New Covid deaths -7,8.47%,47.84%,3.58%,43.53%,25.86%
New Covid cases -7,13.85%,33.90%,6.14%,45.79%,24.92%


Feature importance date pair id lags


,Linear*,KNN*,G-Boost*,MLP*,avg_importance
feature,,,,,
C1 School closing,100.00%,68.53%,81.17%,67.62%,79.33%
C3 Cancel public events,10.14%,100.00%,56.07%,78.14%,61.08%
C7 Restr. Internal movement,5.06%,88.83%,21.74%,100.00%,53.91%
C6 Stay home requirements,60.16%,20.35%,79.28%,54.50%,53.57%
C4 Restrictions gatherings,12.39%,57.33%,100.00%,32.60%,50.58%
H2 Testing policy,11.72%,6.12%,68.63%,53.75%,35.05%
H3 Contact tracing,22.76%,15.45%,37.33%,29.31%,26.21%
C5 Close public transport,24.73%,10.98%,6.81%,31.59%,18.53%
C8 Inter. travel controls,0.43%,7.00%,29.73%,27.82%,16.25%
